In [37]:
!pip install requests

  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)


In [38]:
import pandas as pd
import requests

In [24]:
trainvariants = pd.read_csv('../data/training_variants')
trainvariants.sample(5)

,ID,Gene,Variation,Class
932,932,PDGFRB,R987W,1
2871,2871,BRCA2,R2659K,4
2898,2898,BRCA2,G1529R,6
1695,1695,PMS2,R802*,1
2200,2200,PTEN,C105F,4


In [6]:
traintext = pd.read_csv('../data/training_text',sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [7]:
traintext.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [20]:
trainvariants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         3321 non-null   int64 
 1   Gene       3321 non-null   object
 2   Variation  3321 non-null   object
 3   Class      3321 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 103.9+ KB


In [21]:
traintext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      3321 non-null   int64 
 1   TEXT    3316 non-null   object
dtypes: int64(1), object(1)
memory usage: 52.0+ KB


In [29]:
traintext.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [30]:
trainvariants.Gene.unique()

array(['FAM58A', 'CBL', 'SHOC2', 'TERT', 'DICER1', 'PTPRT', 'RHEB',
       'SHQ1', 'CCND2', 'RAD50', 'CCND3', 'RIT1', 'CCNE1', 'RYBP',
       'TGFBR1', 'TGFBR2', 'MSH6', 'KMT2D', 'LATS1', 'PBRM1', 'SF3B1',
       'LATS2', 'EGFR', 'NKX2-1', 'EIF1AX', 'ARID2', 'BRD4', 'HIST1H1C',
       'ERRFI1', 'CHEK2', 'PAK1', 'TMPRSS2', 'H3F3A', 'ELF3', 'ROS1',
       'ASXL2', 'CDH1', 'EPCAM', 'EP300', 'EPAS1', 'TP53', 'TP53BP1',
       'SMAD2', 'SMAD3', 'SMAD4', 'CDK4', 'AURKB', 'CDK6', 'FBXW7',
       'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'ASXL1',
       'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'BRIP1', 'ERCC3', 'ERCC4',
       'ABL1', 'CEBPA', 'ERG', 'HLA-A', 'HLA-B', 'PDGFRA', 'PDGFRB',
       'RBM10', 'KDM5C', 'ESR1', 'ETV1', 'ETV6', 'TSC1', 'IKZF1', 'TSC2',
       'EWSR1', 'ACVR1', 'FOXA1', 'MDM2', 'ARID1A', 'EZH2', 'MDM4',
       'KLF4', 'RXRA', 'MAP3K1', 'AXIN1', 'BAP1', 'MEN1', 'FANCA',
       'FANCC', 'MET', 'U2AF1', 'KMT2C', 'FAT1', 'ARID1B', 'PIK3CA',
       'PIK3CB', 'PIM1

In [35]:
trainvariants.Variation.sample(20)

853            Y253H
1601            Y98N
1024          P1675L
2747           G466R
2991    E554_K558del
1186           N345I
278            G465E
2320           D544H
746            D821N
671             G23D
2753         Fusions
2998    M552_K558del
60             R453C
2096        Deletion
2597            S59R
2026           N382H
2926            I28T
627            T205A
1884          M2327I
314            K320E
Name: Variation, dtype: object

In [18]:
uvariants = trainvariants.Variation.unique().size
nvariants = trainvariants.Variation.size

print(f'Number of different variants: {uvariants}\nNumber of columns: {nvariants}')

Number of different variants: 2996
Number of columns: 3321


In [33]:
trainvariants.Gene[3218]

'NTRK1'

In [56]:
Description = []
for gen in trainvariants.Gene:
    url = 'https://www.genenetwork.nl/api/v1/gene/' + gen
    dic = requests.get(url).json()
    Description.append(dic['gene']['description'])

In [57]:
trainvariants['Description'] = Description

In [59]:
trainvariants.Description.unique()

array(['family with sequence similarity 58, member A [Source:HGNC Symbol;Acc:HGNC:28434]',
       'Cbl proto-oncogene, E3 ubiquitin protein ligase [Source:HGNC Symbol;Acc:HGNC:1541]',
       'SHOC2 leucine-rich repeat scaffold protein [Source:HGNC Symbol;Acc:HGNC:15454]',
       'telomerase reverse transcriptase [Source:HGNC Symbol;Acc:HGNC:11730]',
       'dicer 1, ribonuclease type III [Source:HGNC Symbol;Acc:HGNC:17098]',
       'protein tyrosine phosphatase, receptor type, T [Source:HGNC Symbol;Acc:HGNC:9682]',
       'Ras homolog enriched in brain [Source:HGNC Symbol;Acc:HGNC:10011]',
       'SHQ1, H/ACA ribonucleoprotein assembly factor [Source:HGNC Symbol;Acc:HGNC:25543]',
       'cyclin D2 [Source:HGNC Symbol;Acc:HGNC:1583]',
       'RAD50 homolog, double strand break repair protein [Source:HGNC Symbol;Acc:HGNC:9816]',
       'cyclin D3 [Source:HGNC Symbol;Acc:HGNC:1585]',
       'Ras-like without CAAX 1 [Source:HGNC Symbol;Acc:HGNC:10023]',
       'cyclin E1 [Source:HGNC Symbo

In [ ]:
dic['gene']['description']